In [369]:
import numpy as np
import xlrd

In [370]:
doc = xlrd.open_workbook('指挥喵/天赋.xlsx')

In [371]:
sheet = doc.sheet_by_index(0)
print(sheet.name, sheet.ncols, sheet.nrows)

Sheet1 18 56


In [372]:
data = []
for i in range(sheet.nrows):
    row = []
    for j in range(sheet.ncols):
        row.append(get_cell(i, j))
    data.append(row)

In [373]:
def parsed(value):
    if isinstance(value, str):
        return value.strip()
    return value
    
def get_cell(i, j):
    for x0, x1, y0, y1 in sheet.merged_cells:
        if x0 == i and y0 == j:
            break
        if x0 <= i < x1 and y0 <= j < y1:
            return parsed(sheet.cell(x0, y0).value)
    return parsed(sheet.cell(i, j).value)


In [374]:
names = data[2]
split_idx = [4, 16]

a, b = split_idx
desc = names[:a]
score = names[a:b]
chance = names[b:]


In [375]:
info = {}
for line in data[3:]:
    if not any(line):
        continue
    item = {
        'name': line[0],
        'desc': dict(zip(names, line[:a])),
        'scores': dict(zip(score, line[a:b])),
        'chance': dict(zip(chance, line[b:])),
    }
    name = item['name']
    if name not in info:
        info[name] = item
    else:
        info[name]['desc'].append(item['desc'])
        for key in score:
            if item['scores'][key]:
                info[name]['scores'][key] = item['scores'][key]
    if name == '':
        print(line)
        
for item in info.values():
    for key in score:
        if item['scores'][key] == '':
            item['scores'][key] = 0


In [376]:
info

{'一发入魂': {'chance': {'SR': 1.0, 'SSR': 1.0},
  'desc': {'天赋名': '一发入魂',
   '描述': '战巡、战列炮击提高15点，主炮暴击率提高3%',
   '特殊天赋': 1.0,
   '简称': '发'},
  'name': '一发入魂',
  'scores': {'战列': 9.0,
   '战巡': 9.0,
   '正航': 0,
   '潜艇': 0,
   '炮巡': 0,
   '维修': 0,
   '航战': 0,
   '轻航': 0,
   '轻雷巡': 0,
   '重炮': 0,
   '重雷巡': 0,
   '驱逐': 0}},
 '不动如山': {'chance': {'SR': 1.0, 'SSR': 1.0},
  'desc': {'天赋名': '不动如山', '描述': '舰队成员受到伤害降低3%', '特殊天赋': 1.0, '简称': '山'},
  'name': '不动如山',
  'scores': {'战列': 6.0,
   '战巡': 6.0,
   '正航': 6.0,
   '潜艇': 2.0,
   '炮巡': 6.0,
   '维修': 6.0,
   '航战': 6.0,
   '轻航': 6.0,
   '轻雷巡': 6.0,
   '重炮': 6.0,
   '重雷巡': 6.0,
   '驱逐': 6.0}},
 '侵略如火': {'chance': {'SR': 1.0, 'SSR': 1.0},
  'desc': {'天赋名': '侵略如火', '描述': '舰队成员造成伤害提高3%', '特殊天赋': 1.0, '简称': '火'},
  'name': '侵略如火',
  'scores': {'战列': 10.0,
   '战巡': 10.0,
   '正航': 10.0,
   '潜艇': 10.0,
   '炮巡': 10.0,
   '维修': 0,
   '航战': 10.0,
   '轻航': 10.0,
   '轻雷巡': 10.0,
   '重炮': 10.0,
   '重雷巡': 10.0,
   '驱逐': 10.0}},
 '其徐如林': {'chance': {'SR': 1.0, 'SSR':

In [400]:
fleet = {
    '驱逐': 3,
    '战列': 3,
}
def sort_talents(fleet, talents=info):
    def key(item):
        item['ScoreInfo'] = []
        score = 0
        for key in fleet:
            score_add = item['scores'][key] * fleet[key]
            if score_add:
                item['ScoreInfo'].append((key, fleet[key], item['scores'][key],))
                score += score_add
        item['Score'] = score
        return -score
    items = list(talents.values())
    items.sort(key=key)
    items = [{'score': item['Score'], 'name': item['name'], 'scoreInfo': item['ScoreInfo']} for item in items if item['Score'] > 0]
    return items

def show_scores(scores):
    for i, item in enumerate(scores):
        print('{1:2d}: {0[score]:2.0f}, {0[name]:}  {0[scoreInfo]}'.format(item, i+1))
    print()
        
        
def show_talents(fleet, talents=info, limit=None):
    print(fleet)
    items = sort_talents(fleet, talents)
    if limit:
        items = items[:limit]
    show_scores(items)

In [378]:
choices = [
    # 道中
    {'驱逐': 3, '维修': 1, '战列': 2},
    {'炮巡': 3, '轻航': 1, '战列': 1, '正航': 1},
    # Boss
    {'驱逐': 3, '战列': 3},
    {'驱逐': 3, '正航': 3},
    {'驱逐': 2, '炮巡':1, '正航': 3},
]

for fleet in choices:
    show_talents(fleet, limit=15)

{'驱逐': 3, '维修': 1, '战列': 2}
 1: 50, 侵略如火  [('驱逐', 3, 10.0), ('战列', 2, 10.0)]
 2: 37, 既定的命运  [('驱逐', 3, 7.0), ('战列', 2, 8.0)]
 3: 36, 不动如山  [('驱逐', 3, 6.0), ('维修', 1, 6.0), ('战列', 2, 6.0)]
 4: 31, 小小的奇迹  [('驱逐', 3, 7.0), ('战列', 2, 5.0)]
 5: 30, 其徐如林  [('驱逐', 3, 6.0), ('战列', 2, 6.0)]
 6: 27, 水雷魂  [('驱逐', 3, 9.0)]
 7: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 8: 21, 新人雷击士·驱逐  [('驱逐', 3, 7.0)]
 9: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]
10: 18, 一发入魂  [('战列', 2, 9.0)]
11: 18, 新晋指挥官·铁血  [('驱逐', 3, 4.0), ('战列', 2, 3.0)]
12: 17, 新晋指挥官·皇家  [('驱逐', 3, 3.0), ('战列', 2, 4.0)]
13: 16, 炮击新手·主力  [('战列', 2, 8.0)]
14: 16, 新晋指挥官·重樱  [('驱逐', 3, 4.0), ('战列', 2, 2.0)]
15: 15, 炮击新手·驱逐  [('驱逐', 3, 5.0)]

{'炮巡': 3, '轻航': 1, '战列': 1, '正航': 1}
 1: 60, 侵略如火  [('炮巡', 3, 10.0), ('轻航', 1, 10.0), ('战列', 1, 10.0), ('正航', 1, 10.0)]
 2: 44, 既定的命运  [('炮巡', 3, 8.0), ('轻航', 1, 6.0), ('战列', 1, 8.0), ('正航', 1, 6.0)]
 3: 36, 不动如山  [('炮巡', 3, 6.0), ('轻航', 1, 6.0), ('战列', 1, 6.0), ('正航', 1, 6.0)]
 4: 33, 其徐如林  [('炮巡', 3, 5.0), ('轻航', 1, 6.0), ('战列'

In [379]:
show_talents({'炮巡': 3, '战列': 2, '轻航': 1}, limit=15)

{'炮巡': 3, '战列': 2, '轻航': 1}
 1: 60, 侵略如火  [('炮巡', 3, 10.0), ('战列', 2, 10.0), ('轻航', 1, 10.0)]
 2: 46, 既定的命运  [('炮巡', 3, 8.0), ('战列', 2, 8.0), ('轻航', 1, 6.0)]
 3: 36, 不动如山  [('炮巡', 3, 6.0), ('战列', 2, 6.0), ('轻航', 1, 6.0)]
 4: 33, 其徐如林  [('炮巡', 3, 5.0), ('战列', 2, 6.0), ('轻航', 1, 6.0)]
 5: 24, 见敌必战  [('炮巡', 3, 8.0)]
 6: 21, 炮击新手·巡洋  [('炮巡', 3, 7.0)]
 7: 21, 新晋指挥官·巡洋  [('炮巡', 3, 7.0)]
 8: 21, 新晋指挥官·白鹰  [('炮巡', 3, 3.0), ('战列', 2, 3.0), ('轻航', 1, 6.0)]
 9: 21, 新晋指挥官·皇家  [('炮巡', 3, 4.0), ('战列', 2, 4.0), ('轻航', 1, 1.0)]
10: 20, 新晋指挥官·铁血  [('炮巡', 3, 4.0), ('战列', 2, 3.0), ('轻航', 1, 2.0)]
11: 18, 小小的奇迹  [('炮巡', 3, 2.0), ('战列', 2, 5.0), ('轻航', 1, 2.0)]
12: 18, 一发入魂  [('战列', 2, 9.0)]
13: 17, 新晋指挥官·重樱  [('炮巡', 3, 3.0), ('战列', 2, 2.0), ('轻航', 1, 4.0)]
14: 16, 炮击新手·主力  [('战列', 2, 8.0)]
15: 15, 其疾如风  [('炮巡', 3, 2.0), ('战列', 2, 3.0), ('轻航', 1, 3.0)]



In [380]:
show_talents({'驱逐': 3})

{'驱逐': 3}
 1: 30, 侵略如火  [('驱逐', 3, 10.0)]
 2: 27, 水雷魂  [('驱逐', 3, 9.0)]
 3: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 4: 21, 小小的奇迹  [('驱逐', 3, 7.0)]
 5: 21, 既定的命运  [('驱逐', 3, 7.0)]
 6: 21, 新人雷击士·驱逐  [('驱逐', 3, 7.0)]
 7: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]
 8: 18, 其徐如林  [('驱逐', 3, 6.0)]
 9: 18, 不动如山  [('驱逐', 3, 6.0)]
10: 15, 炮击新手·驱逐  [('驱逐', 3, 5.0)]
11: 15, 装填新手·驱逐  [('驱逐', 3, 5.0)]
12: 12, 新晋指挥官·重樱  [('驱逐', 3, 4.0)]
13: 12, 新晋指挥官·铁血  [('驱逐', 3, 4.0)]
14:  9, 新晋指挥官·白鹰  [('驱逐', 3, 3.0)]
15:  9, 新晋指挥官·皇家  [('驱逐', 3, 3.0)]
16:  3, 其疾如风  [('驱逐', 3, 1.0)]
17:  3, 对空炮手·先锋  [('驱逐', 3, 1.0)]
18:  3, 声纳兵·先锋  [('驱逐', 3, 1.0)]
19:  3, 新手观测士·先锋  [('驱逐', 3, 1.0)]
20:  3, 操舵手·小型舰  [('驱逐', 3, 1.0)]



In [381]:
show_talents({'驱逐': 3, '战列': 0}, limit=15)

{'驱逐': 3, '战列': 0}
 1: 30, 侵略如火  [('驱逐', 3, 10.0)]
 2: 27, 水雷魂  [('驱逐', 3, 9.0)]
 3: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 4: 21, 小小的奇迹  [('驱逐', 3, 7.0)]
 5: 21, 既定的命运  [('驱逐', 3, 7.0)]
 6: 21, 新人雷击士·驱逐  [('驱逐', 3, 7.0)]
 7: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]
 8: 18, 其徐如林  [('驱逐', 3, 6.0)]
 9: 18, 不动如山  [('驱逐', 3, 6.0)]
10: 15, 炮击新手·驱逐  [('驱逐', 3, 5.0)]
11: 15, 装填新手·驱逐  [('驱逐', 3, 5.0)]
12: 12, 新晋指挥官·重樱  [('驱逐', 3, 4.0)]
13: 12, 新晋指挥官·铁血  [('驱逐', 3, 4.0)]
14:  9, 新晋指挥官·白鹰  [('驱逐', 3, 3.0)]
15:  9, 新晋指挥官·皇家  [('驱逐', 3, 3.0)]



In [382]:
# 完美队伍
def get_score(fleet, talents):
    fleet

In [383]:
talents = ['装填新手·战列', '轮机手·战列', '见敌必战']
# talents = ['炮击新手·驱逐', '新晋指挥官·驱逐', '轮机手·驱逐']
talents = {key: info[key] for key in talents}
scores = sort_talents({'炮巡': 1, '轻雷巡': 2, '战列': 3}, talents)
sum([item['score'] for item in scores]), scores

(48.0,
 [{'name': '见敌必战',
   'score': 24.0,
   'scoreInfo': [('炮巡', 1, 8.0), ('轻雷巡', 2, 8.0)]},
  {'name': '装填新手·战列', 'score': 18.0, 'scoreInfo': [('战列', 3, 6.0)]},
  {'name': '轮机手·战列', 'score': 6.0, 'scoreInfo': [('战列', 3, 2.0)]}])

In [395]:
talents = ['既定的命运', '操舵手·小型舰']
# talents = ['炮击新手·驱逐', '新晋指挥官·驱逐', '轮机手·驱逐']
talents = {key: info[key] for key in talents}
scores = sort_talents({'炮巡': 1, '驱逐': 2, '战列': 3}, talents)
print(sum([item['score'] for item in scores]), scores)

scores = sort_talents({'炮巡': 1, '轻雷巡': 2, '战列': 3}, talents)
print(sum([item['score'] for item in scores]), scores)

48.0 [{'score': 46.0, 'name': '既定的命运', 'scoreInfo': [('炮巡', 1, 8.0), ('驱逐', 2, 7.0), ('战列', 3, 8.0)]}, {'score': 2.0, 'name': '操舵手·小型舰', 'scoreInfo': [('驱逐', 2, 1.0)]}]
48.0 [{'score': 48.0, 'name': '既定的命运', 'scoreInfo': [('炮巡', 1, 8.0), ('轻雷巡', 2, 8.0), ('战列', 3, 8.0)]}]


In [385]:
show_talents({'正航': 2, '轻航': 1}, limit=15)

{'正航': 2, '轻航': 1}
 1: 30, 侵略如火  [('正航', 2, 10.0), ('轻航', 1, 10.0)]
 2: 21, 航空新兵·空母  [('正航', 2, 7.0), ('轻航', 1, 7.0)]
 3: 18, 其徐如林  [('正航', 2, 6.0), ('轻航', 1, 6.0)]
 4: 18, 不动如山  [('正航', 2, 6.0), ('轻航', 1, 6.0)]
 5: 18, 既定的命运  [('正航', 2, 6.0), ('轻航', 1, 6.0)]
 6: 18, 新晋指挥官·白鹰  [('正航', 2, 6.0), ('轻航', 1, 6.0)]
 7: 16, 新晋指挥官·空母  [('正航', 2, 8.0)]
 8: 14, 王牌机师  [('正航', 2, 7.0)]
 9: 12, 新手整备士  [('正航', 2, 4.0), ('轻航', 1, 4.0)]
10: 12, 新晋指挥官·重樱  [('正航', 2, 4.0), ('轻航', 1, 4.0)]
11:  9, 其疾如风  [('正航', 2, 3.0), ('轻航', 1, 3.0)]
12:  9, 新手观测士·主力  [('正航', 2, 3.0), ('轻航', 1, 3.0)]
13:  6, 小小的奇迹  [('正航', 2, 2.0), ('轻航', 1, 2.0)]
14:  6, 对空炮手·主力  [('正航', 2, 2.0), ('轻航', 1, 2.0)]
15:  6, 轮机手·空母  [('正航', 2, 2.0), ('轻航', 1, 2.0)]



In [386]:
short_dict = {
    '航': {'正航', '轻航'},
    '战': {'战列', '战巡', '航战'},
    '驱': {'驱逐'},
    '巡': {'轻雷巡', '重雷巡', '炮巡'},
    '潜': {'潜艇'}
}

def search_talent(ship_types, talent_short):
    candidates = []
    for talent in info.values():
        if talent_short == talent['desc']['简称']:
            candidates.append(talent)
    if len(candidates) == 1:
        return candidates[0]['name']
    for talent in candidates:
        for ship_type in ship_types:
            if talent['scores'][ship_type] > 0:
                return talent['name']
    return {"Error": {'ship_types': ship_types, 'talent_short': talent_short}, 'candidates': candidates}
    
def get_talents(cat):
    s_type, s_talents = cat.split(' ')
    ship_types = short_dict[s_type]
    
    talents = []
    for s in s_talents:
        talents.append(search_talent(ship_types, s))
    return talents

In [387]:
get_talents('航 航樱')

['航空新兵·空母', '新晋指挥官·重樱']

In [353]:
get_talents('战 炮')

['炮击新手·主力']

In [354]:
get_talents('战 指空山')

['新晋指挥官·战列', '对空炮手·主力', '不动如山']

In [355]:
get_talents('驱 指耐装')

['新晋指挥官·驱逐', '轮机手·驱逐', '装填新手·驱逐']

In [391]:
get_talents('驱 运机')

['既定的命运', '操舵手·小型舰']

In [390]:
get_talents('驱 运机')

['既定的命运', '操舵手·小型舰']

In [368]:
t.strip()

'既定的命运'

In [410]:
def cat_score(fleet, cat):
    talents = {key: info[key] for key in get_talents(cat)}
    scores = sort_talents(fleet, talents)[:15]
    total_score = sum([item['score'] for item in scores])
    print('{:3.0f}: {} for {}'.format(total_score, cat, fleet, ))
    show_scores(scores)

In [411]:
cat_score({'驱逐': 3, '战列': 3}, '驱 运机')

 48: 驱 运机 for {'驱逐': 3, '战列': 3}
 1: 45, 既定的命运  [('驱逐', 3, 7.0), ('战列', 3, 8.0)]
 2:  3, 操舵手·小型舰  [('驱逐', 3, 1.0)]



In [426]:
cats = [
    '战 指空山',
    '战 炮',
    '战 炮皇',
    '战 指耐空',
    '战 炮装',
]
for cat in cats:
    cat_score({'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}, cat)

 39: 战 指空山 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
 1: 30, 不动如山  [('驱逐', 1, 6.0), ('轻雷巡', 2, 6.0), ('轻航', 1, 6.0), ('战列', 1, 6.0)]
 2:  5, 新晋指挥官·战列  [('战列', 1, 5.0)]
 3:  4, 对空炮手·主力  [('轻航', 1, 2.0), ('战列', 1, 2.0)]

  8: 战 炮 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
 1:  8, 炮击新手·主力  [('战列', 1, 8.0)]

 24: 战 炮皇 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
 1: 16, 新晋指挥官·皇家  [('驱逐', 1, 3.0), ('轻雷巡', 2, 4.0), ('轻航', 1, 1.0), ('战列', 1, 4.0)]
 2:  8, 炮击新手·主力  [('战列', 1, 8.0)]

 11: 战 指耐空 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
 1:  5, 新晋指挥官·战列  [('战列', 1, 5.0)]
 2:  4, 对空炮手·主力  [('轻航', 1, 2.0), ('战列', 1, 2.0)]
 3:  2, 轮机手·战列  [('战列', 1, 2.0)]

 14: 战 炮装 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
 1:  8, 炮击新手·主力  [('战列', 1, 8.0)]
 2:  6, 装填新手·战列  [('战列', 1, 6.0)]

 24: 巡 雷炮 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
 1: 12, 新人雷击士·巡洋  [('轻雷巡', 2, 6.0)]
 2: 12, 炮击新手·巡洋  [('轻雷巡', 2, 6.0)]



In [427]:
cats = [
    '巡 雷炮',
    '巡 雷指',
    '巡 炮皇',
]
for cat in cats:
    cat_score({'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}, cat)

 24: 巡 雷炮 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
 1: 12, 新人雷击士·巡洋  [('轻雷巡', 2, 6.0)]
 2: 12, 炮击新手·巡洋  [('轻雷巡', 2, 6.0)]

 26: 巡 雷指 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
 1: 14, 新晋指挥官·巡洋  [('轻雷巡', 2, 7.0)]
 2: 12, 新人雷击士·巡洋  [('轻雷巡', 2, 6.0)]

 28: 巡 炮皇 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
 1: 16, 新晋指挥官·皇家  [('驱逐', 1, 3.0), ('轻雷巡', 2, 4.0), ('轻航', 1, 1.0), ('战列', 1, 4.0)]
 2: 12, 炮击新手·巡洋  [('轻雷巡', 2, 6.0)]



In [430]:
cats = [
    '驱 指耐装',
    '驱 雷机指',
    '驱 耐装炮中',
    '驱 指炮中',
]
for cat in cats:
    cat_score({'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}, cat)

 40: 驱 指耐装 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 16, 新晋指挥官·驱逐  [('驱逐', 2, 8.0)]
 2: 14, 轮机手·驱逐  [('驱逐', 2, 7.0)]
 3: 10, 装填新手·驱逐  [('驱逐', 2, 5.0)]

 32: 驱 雷机指 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 16, 新晋指挥官·驱逐  [('驱逐', 2, 8.0)]
 2: 14, 新人雷击士·驱逐  [('驱逐', 2, 7.0)]
 3:  2, 操舵手·小型舰  [('驱逐', 2, 1.0)]

 38: 驱 耐装炮中 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 14, 轮机手·驱逐  [('驱逐', 2, 7.0)]
 2: 10, 装填新手·驱逐  [('驱逐', 2, 5.0)]
 3: 10, 炮击新手·驱逐  [('驱逐', 2, 5.0)]
 4:  4, 新手观测士·先锋  [('驱逐', 2, 1.0), ('炮巡', 1, 2.0)]

 30: 驱 指炮中 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 16, 新晋指挥官·驱逐  [('驱逐', 2, 8.0)]
 2: 10, 炮击新手·驱逐  [('驱逐', 2, 5.0)]
 3:  4, 新手观测士·先锋  [('驱逐', 2, 1.0), ('炮巡', 1, 2.0)]



In [431]:
cats = [
    '航 航樱',
    '航 樱指潜',
    '航 耐中指',
    '航 装机航',
]
for cat in cats:
    cat_score({'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}, cat)

 35: 航 航樱 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 21, 新晋指挥官·重樱  [('驱逐', 2, 4.0), ('炮巡', 1, 3.0), ('正航', 2, 4.0), ('战列', 1, 2.0)]
 2: 14, 航空新兵·空母  [('正航', 2, 7.0)]

 37: 航 樱指潜 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 21, 新晋指挥官·重樱  [('驱逐', 2, 4.0), ('炮巡', 1, 3.0), ('正航', 2, 4.0), ('战列', 1, 2.0)]
 2: 16, 新晋指挥官·空母  [('正航', 2, 8.0)]

 32: 航 耐中指 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 16, 新晋指挥官·空母  [('正航', 2, 8.0)]
 2: 12, 新手观测士·主力  [('正航', 2, 3.0), ('战列', 1, 6.0)]
 3:  4, 轮机手·空母  [('正航', 2, 2.0)]

 24: 航 装机航 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 14, 航空新兵·空母  [('正航', 2, 7.0)]
 2:  8, 新手整备士  [('正航', 2, 4.0)]
 3:  2, 操舵手·中型舰  [('炮巡', 1, 2.0)]



In [446]:
cats = [
    '驱 耐指',
    
    '驱 指耐装',
    '驱 雷机指',
    '驱 耐装炮中',
    '驱 指炮中',
    
    '战 指空山',
    '战 炮',
    '战 炮皇',
    '战 指耐空',
    '战 炮装',
    
    
    '航 航樱',
    '航 航装',
    '航 樱指潜',
    '航 耐中指',
    '航 装机航',
]
for cat in cats:
    cat_score({'驱逐': 3, '轻航': 1, '战列': 1}, cat)

 45: 驱 耐指 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 2: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]

 60: 驱 指耐装 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 2: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]
 3: 15, 装填新手·驱逐  [('驱逐', 3, 5.0)]

 48: 驱 雷机指 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 2: 21, 新人雷击士·驱逐  [('驱逐', 3, 7.0)]
 3:  3, 操舵手·小型舰  [('驱逐', 3, 1.0)]

 54: 驱 耐装炮中 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]
 2: 15, 装填新手·驱逐  [('驱逐', 3, 5.0)]
 3: 15, 炮击新手·驱逐  [('驱逐', 3, 5.0)]
 4:  3, 新手观测士·先锋  [('驱逐', 3, 1.0)]

 42: 驱 指炮中 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 2: 15, 炮击新手·驱逐  [('驱逐', 3, 5.0)]
 3:  3, 新手观测士·先锋  [('驱逐', 3, 1.0)]

 39: 战 指空山 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1: 30, 不动如山  [('驱逐', 3, 6.0), ('轻航', 1, 6.0), ('战列', 1, 6.0)]
 2:  5, 新晋指挥官·战列  [('战列', 1, 5.0)]
 3:  4, 对空炮手·主力  [('轻航', 1, 2.0), ('战列', 1, 2.0)]

  8: 战 炮 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1:  8, 炮击新手·主力  [('战列', 1, 8.0)]

In [445]:
cats = [
    '驱 耐指',
    
    '驱 指耐装',
    '驱 雷机指',
    '驱 耐装炮中',
    '驱 指炮中雷',
    
    '战 指空山',
    '战 炮',
    '战 炮皇',
    '战 指耐空',
    '战 炮装',
    
    
    '航 航樱',
    '航 航装',
    '航 樱指潜',
    '航 耐中指',
    '航 装机航',
]
for cat in cats:
    cat_score({'驱逐': 3}, cat)


 45: 驱 耐指 for {'驱逐': 3}
 1: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 2: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]

 60: 驱 指耐装 for {'驱逐': 3}
 1: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 2: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]
 3: 15, 装填新手·驱逐  [('驱逐', 3, 5.0)]

 48: 驱 雷机指 for {'驱逐': 3}
 1: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 2: 21, 新人雷击士·驱逐  [('驱逐', 3, 7.0)]
 3:  3, 操舵手·小型舰  [('驱逐', 3, 1.0)]

 54: 驱 耐装炮中 for {'驱逐': 3}
 1: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]
 2: 15, 装填新手·驱逐  [('驱逐', 3, 5.0)]
 3: 15, 炮击新手·驱逐  [('驱逐', 3, 5.0)]
 4:  3, 新手观测士·先锋  [('驱逐', 3, 1.0)]

 63: 驱 指炮中雷 for {'驱逐': 3}
 1: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 2: 21, 新人雷击士·驱逐  [('驱逐', 3, 7.0)]
 3: 15, 炮击新手·驱逐  [('驱逐', 3, 5.0)]
 4:  3, 新手观测士·先锋  [('驱逐', 3, 1.0)]

 18: 战 指空山 for {'驱逐': 3}
 1: 18, 不动如山  [('驱逐', 3, 6.0)]

  0: 战 炮 for {'驱逐': 3}

  9: 战 炮皇 for {'驱逐': 3}
 1:  9, 新晋指挥官·皇家  [('驱逐', 3, 3.0)]

  0: 战 指耐空 for {'驱逐': 3}

  0: 战 炮装 for {'驱逐': 3}

 12: 航 航樱 for {'驱逐': 3}
 1: 12, 新晋指挥官·重樱  [('驱逐', 3, 4.0)]

  0: 航 航装 for {'驱逐': 3}

 12: 航 樱指潜 for {'驱逐': 3}
 1: 12, 新晋指挥官·重樱  

In [444]:
cats = [
    '潜 指中',
    '潜 装',
    '驱 运机',
    '驱 指耐',
]
for cat in cats:
    cat_score({'潜艇': 3}, cat)

 36: 潜 指中 for {'潜艇': 3}
 1: 27, 新晋指挥官·潜艇  [('潜艇', 3, 9.0)]
 2:  9, 新手观测士·潜艇  [('潜艇', 3, 3.0)]

  9: 潜 装 for {'潜艇': 3}
 1:  9, 装填新手·潜艇  [('潜艇', 3, 3.0)]

 24: 驱 运机 for {'潜艇': 3}
 1: 18, 既定的命运  [('潜艇', 3, 6.0)]
 2:  6, 操舵手·小型舰  [('潜艇', 3, 2.0)]

  0: 驱 指耐 for {'潜艇': 3}



In [473]:
items= """
数量	潜艇	驱逐	轻雷巡	重雷巡	炮巡	轻航	正航	战列	战巡	航战	重炮	维修
白鹰		3										
皇家												
重樱						1		1				
铁血												
其他												
"""

data = [line.split('\t') for line in items.strip('\r\n').splitlines()]
nrows, ncols = len(data), len(data[0])

In [474]:
for j in range(ncols):
    for i in range(nrows):
        try:
            print(data[i][j], end='\t')
        except:
            print(i,j)
    print()

数量	白鹰	皇家	重樱	铁血	其他	
潜艇						
驱逐	3					
轻雷巡						
重雷巡						
炮巡						
轻航			1			
正航						
战列			1			
战巡						
航战						
重炮						
维修						
